In [1]:
# Implements the Doubly robust value estimator for the learned policies

In [285]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [286]:
shaped_reward_df = pd.read_csv('../sepsisrl_test_reward_shape/test_with_rewards.csv')

In [287]:
df_train = pd.read_csv('rl_train_data_final.csv')
df_train.head()

,Unnamed: 0,bloc,icustayid,state,reward,max_dose_vaso,iv_tev_in,mortality
0,0,1,3,1128,0,0.0,4.0,0
1,1,2,3,1128,0,0.0,4.0,0
2,2,3,3,1172,0,0.0,2.0,0
3,3,4,3,1172,0,0.0,2.0,0
4,4,5,3,464,0,0.0,2.0,0


In [288]:
df_test =pd.read_csv('rl_test_data_final.csv')

In [289]:
phys_data = pickle.load(open("phys_policy.p", "rb" ))

In [290]:
pd.DataFrame(phys_data).head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,14.0,1.0,4.0,0.0,20.0,0.001133,4.817572e-09,2.745525e-09,8.324157e-10,4.582567e-13,...,0.239332,4.335571e-08,5.368753e-06,1.702110e-09,2.096259e-10,0.299623,1.106430e-07,3.070747e-06,2.124976e-06,9.804121e-06
1,14.0,2.0,3.0,0.0,15.0,0.003760,7.312874e-09,8.960193e-09,1.365068e-09,1.747765e-11,...,0.436484,7.530244e-09,4.418755e-07,9.701403e-10,3.632597e-10,0.163057,4.214060e-08,4.274639e-07,8.589486e-08,3.796162e-06
2,14.0,3.0,3.0,0.0,15.0,0.000829,1.082430e-07,8.162170e-09,1.298296e-09,7.458415e-11,...,0.237408,5.900292e-08,6.229191e-07,2.191570e-09,1.585154e-10,0.415724,9.835096e-07,1.030690e-06,1.291592e-07,2.353789e-07
3,14.0,4.0,2.0,0.0,10.0,0.001118,6.395420e-08,1.997361e-08,4.802769e-10,1.819435e-10,...,0.500454,1.379003e-07,6.040940e-07,1.911497e-09,2.186019e-09,0.158869,8.406097e-07,3.160200e-07,6.267963e-09,3.789125e-08
4,14.0,5.0,0.0,0.0,0.0,0.575546,3.755423e-08,9.059047e-06,2.401201e-09,5.102324e-09,...,0.016134,1.618260e-05,6.457800e-06,8.623400e-07,5.999016e-06,0.017737,4.673912e-05,5.967060e-05,1.357733e-06,1.167808e-05


In [291]:
indices = phys_data[:,4].astype(int) + 5
phys_action_probs = phys_data[range(len(phys_data)),indices]

In [292]:
# Two cells below - load the actions and q values associated with the agent policy - these are outputted by
# eg the neural network.

In [334]:
agent_actions = pickle.load(open("../sepsis_deeprl2/deeprl_actions_s897_2.p", "rb" ))

In [335]:
agent_q = pickle.load(open("../sepsis_deeprl2/deeprl_qvals_s897_2.p", "rb" ))

In [336]:
# add the actions and q values associated with agent performance on the test set to the dataframe

In [337]:
df_test['agent_action'] = np.array(agent_actions)

In [338]:
df_test['agent_q'] = np.array(agent_q)

In [339]:
df_test['phys_prob'] = phys_action_probs

In [340]:
# # this represents the physician distribution, learned by averaging counts on the training set
# pi_phys = np.ones(shape=(1250,25))

In [341]:
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [342]:
# # calculate physician policy distribution
# for i in df_train.index:
#     state = df_train.ix[i,'state']
#     iv = df_train.ix[i,'iv_tev_in']
#     vaso = df_train.ix[i, 'max_dose_vaso']
#     action = action_map[(iv,vaso)]
#     pi_phys[state,action] += 1.0

# for row in range(pi_phys.shape[0]):
#     total = np.sum(pi_phys[row])
#     pi_phys[row] /= float(total)

In [343]:
df_test['reward'] = shaped_reward_df['reward']

In [344]:
unique_ids = df_test['icustayid'].unique()

In [345]:
values = []
gamma = 1
for uid in unique_ids:
#     if uid != 8298: continue
#     if uid in bad_uids:
#         continue
    trajectory = df_test.loc[df_test['icustayid'] == uid]
    cur_val = 0
    reversed_traj = trajectory.iloc[::-1]
    t = len(reversed_traj)
    for row in reversed_traj.index:
        state = df_test.ix[row,'state']
        
        iv = df_test.ix[row,'iv_tev_in']
        vaso = df_test.ix[row, 'max_dose_vaso']
        phys_action = action_map[(iv,vaso)]
        phys_prob = df_test.ix[row,'phys_prob']
        
#         phys_prob = pi_phys[state,phys_action]
        
        agent_action = reversed_traj.ix[row,'agent_action']
#         agent_action = phys_action
        agent_val = reversed_traj.ix[row, 'agent_q']
        
        reward = reversed_traj.ix[row, 'reward']
#         if reward > 15:
#             reward = 15
#         elif reward < -15:
#             reward = -15
        
#         rho = 1.0
        if agent_action == phys_action:
            rho = 1.0/phys_prob
        else:
            rho = 0
        
#         # Debug related things below
#         print "row ", row
#         print "phys_prob ", phys_prob
#         print "state", state
#         print "actions ", agent_action, phys_action
#         print "agent val ", agent_val
#         print "cur val before update  ", cur_val
        
        
        cur_val = agent_val + rho*(reward + gamma*cur_val - agent_val)
        t -= 1
    
#        ~ More debugging below
#         print "cur val after update  ", cur_val
#         print "rho ", rho
#         print "reward ", reward

    assert (t == 0)

#     if abs(cur_val) > 1000:
#         print uid
#         bad_uids.append(uid)
    values.append(cur_val)

In [346]:
len(values)

3580

In [347]:
values = pd.Series(values)
values = (values[(values >= -15) & (values <= 15)])

In [348]:
len(values)

3443

In [349]:
np.mean(values)

9.7414900121408685

In [310]:
# # for use when rewards are sparse and also +-100 instead of +-15
# df_test.loc[df_test['reward'] == 100,'reward'] = 15
# df_test.loc[df_test['reward'] == -100,'reward'] = -15

In [312]:
# #  The code here evaluates the expected return under the physician policy
# phys_vals = []
# for uid in unique_ids:
#     traj = df_test.loc[df_test['icustayid'] == uid]
#     phys_vals.append(sum(traj['reward']))
# np.mean(phys_vals)

11.170391061452515